In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
import pickle
import myUtils
import os
import csv
import matplotlib.pyplot as plt
from myUtils import *
from models.utils import *
from score import ScoreUtils
from models.dataset_loader import *
import sys
sys.path.append('models')

warnings.simplefilter("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
dataset_name = 'toy-spam'
model_type = 'logistic'
model_name = 'huawei-noah/TinyBERT_General_4L_312D'
sorting = 'confidence'
delta = '0.1'
folder_name = f'results/mp/{model_type}/{dataset_name}/{sorting}/42/stop-words-0.1'    

In [4]:
trail_path='/sum_scores.xlsx'
df = pd.read_excel(f'{folder_name}{trail_path}', skiprows=[0]).loc[:, 'name':]
df = df.sort_values(by=['normal'], ascending=False)
df = df[df['type occurences'] > 10]
df = df[:10].sort_values(by=['anchor score'], ascending=False)
df

,name,anchor score,type occurences,total occurences,+%,-%,both,normal,Unnamed: 8,name.1,anchor score.1,type occurences.1,total occurences.1,+%.1,-%.1,both.1,normal.1
2,.,0.029820,342,357,0.04,0.96,True,9290,NaN,loves,0.078324,129.0,129.0,1.00,0.00,0.0,353.0
4,it,0.026768,307,320,0.04,0.96,True,6091,NaN,loved,0.044323,73.0,73.0,1.00,0.00,0.0,254.0
5,the,0.021188,243,248,0.02,0.98,True,6098,NaN,fun,0.041287,68.0,68.0,1.00,0.00,0.0,1073.0
11,",",0.010376,119,124,0.04,0.96,True,3502,NaN,excellent,0.014572,24.0,24.0,1.00,0.00,0.0,40.0
13,i,0.009504,109,112,0.03,0.97,True,3988,NaN,perfect,0.011536,19.0,19.0,1.00,0.00,0.0,147.0
16,to,0.009068,104,106,0.02,0.98,True,3943,NaN,and,0.009107,15.0,72.0,0.21,0.79,1.0,4424.0
21,a,0.007237,83,86,0.03,0.97,True,4177,NaN,wonderful,0.007893,13.0,13.0,1.00,0.00,0.0,43.0
24,for,0.006539,75,76,0.01,0.99,True,3304,NaN,friends,0.007286,12.0,12.0,1.00,0.00,0.0,177.0
29,this,0.005755,66,75,0.12,0.88,True,3477,NaN,lego,0.006072,10.0,10.0,1.00,0.00,0.0,60.0
34,and,0.004970,57,72,0.21,0.79,True,4424,NaN,this,0.005464,9.0,75.0,0.12,0.88,1.0,3477.0


In [5]:
score_indices = df['name']

In [4]:
trail_path='/scores.xlsx'
df = pd.read_excel(f'{folder_name}{trail_path}', skiprows=[0]).loc[:, 'name':]

In [7]:
trail_path='/sum_scores.xlsx'
df2 = pd.read_excel(f'{folder_name}{trail_path}', skiprows=[0]).loc[:, 'name':]

In [29]:
prob_names = df['name.7'][:20]
sum_names = df2['name.1'][:20]
prob_indices = []
sum_indices = []
for i in range(20):
    prob_indices.append(df.index[df['name.7'] == sum_names[i]].values[0])
    sum_indices.append(df2.index[df2['name.1'] == prob_names[i]].values[0])

In [31]:
for w, w2, i, i2 in zip(prob_names, sum_names, prob_indices, sum_indices):
    print(f'{w} & {w2} & {i} & {i2} \\\ \hline')

hand & hand & 0 & 0 \\ \hline
like & like & 1 & 1 \\ \hline
help & help & 2 & 2 \\ \hline
good & good & 3 & 3 \\ \hline
safe & please & 5 & 5 \\ \hline
please & safe & 4 & 4 \\ \hline
thank & thank & 6 & 6 \\ \hline
great & great & 7 & 7 \\ \hline
free & 19 & 48 & 9 \\ \hline
support & free & 8 & 11 \\ \hline
thanks & co & 751 & 12 \\ \hline
well & support & 9 & 13 \\ \hline
best & thanks & 10 & 15 \\ \hline
positive & well & 11 & 18 \\ \hline
better & corona & 752 & 21 \\ \hline
care & best & 12 & 20 \\ \hline
love & store & 33 & 22 \\ \hline
safety & grocery & 42 & 25 \\ \hline
relief & positive & 13 & 26 \\ \hline
interesting & shopping & 26 & 30 \\ \hline


In [6]:
for x in list(df['name.7'][:10]):
    print(x+',', end=' ')

hand, like, help, good, safe, thank, great, free, support, thanks, 

In [6]:
print(list(df['name.7'][:10]), end=' ')

['great', 'love', 'loves', 'game', 'loved', 'awesome', 'fun', 'highly', 'favorite', 'family'] 

In [113]:
trail_path='/scores.xlsx'
columns = ['name', 'anchor score', 'type occurences', 'total occurences', '+%', '-%', 'both', 'normal']
df = pd.read_excel(f'{folder_name}{trail_path}').drop(0)
c = list(df.columns).index('0.5-negative')
df = df.iloc[:,c:c+8]
df.columns = columns
df = df.sort_values(by=['normal'], ascending=False)
df = df[df['type occurences'] > 10]
df = df[:10].sort_values(by=['anchor score'], ascending=False)
df

,name,anchor score,type occurences,total occurences,+%,-%,both,normal
5,would,0.002618,115,115,0,1,False,559
13,product,0.001606,62,62,0,1,False,392
47,like,0.000984,40,40,0,1,False,568
96,little,0.000822,27,27,0,1,False,422
120,buy,0.000787,22,22,0,1,False,333
334,much,0.000658,19,23,0.17,0.83,True,411
1474,get,0.000547,14,15,0.07,0.93,True,414
1503,time,0.000465,11,13,0.15,0.85,True,434
1504,one,0.000451,23,27,0.15,0.85,True,767
1506,toy,0.000427,17,17,0,1,False,638


In [ ]:
for x in list(df['name']):
    print(x+',', end=' ')

In [114]:
for i, i2, w in zip(score_indices.index, df.index, df['name']):
    print(f'{i} & {i2} & {w} \\\ \hline')

3 & 5 & would \\ \hline
7 & 13 & product \\ \hline
19 & 47 & like \\ \hline
32 & 96 & little \\ \hline
42 & 120 & buy \\ \hline
46 & 334 & much \\ \hline
59 & 1474 & get \\ \hline
71 & 1503 & time \\ \hline
89 & 1504 & one \\ \hline
115 & 1506 & toy \\ \hline


In [117]:
model = load_model(f'models/{model_type}/{dataset_name}/traced.pt').to(device).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
myUtils.model = model
myUtils.tokenizer = tokenizer

In [12]:
exps = pickle.load(open(f"{folder_name}/exps_list.pickle", "rb" ))
anchor_examples = pickle.load(open(f"{folder_name}/anchor_examples.pickle", "rb" ))
print(f'anchor examples len: {len(anchor_examples)}')
print(f'exps len: {len(exps)}')

if not os.path.exists(f"{folder_name}/predictions.pickle"):
    predictions = [predict_sentences([tokenizer.tokenize(example)])[0] for example in anchor_examples]
    pickle.dump(predictions, open(f"{folder_name}/predictions.pickle", "wb" ))
    
predictions = pickle.load(open(f"{folder_name}/predictions.pickle", "rb" ))
labels = pickle.load(open(f"{folder_name}/labels.pickle", "rb" ))

anchor examples len: 6110
exps len: 5678


In [118]:
ds = get_ds(dataset_name)['train']
texts, labels = ds['text'], ds['label']

In [119]:
pos_anchors = list(pd.read_excel(f"{folder_name}/scores.xlsx").drop(0)['0.5-positive'].values)
neg_anchors = list(pd.read_excel(f"{folder_name}/scores.xlsx").drop(0)['0.5-negative'].values)

In [120]:
def count(word, target, filter_words=None):
    c=0
    for t, l in zip(texts, labels):
        if l==target and word in t:
            if filter_words is None or any(re.search(r'\b' + w + r'\b', t) for w in filter_words):
                c+=1
    return c

# Corona

In [165]:
count('would', 0)

973

In [154]:
count('hand', 0, neg_anchors[:30])

105

In [155]:
sum(count(w, 0) for w in neg_anchors[:50])

6503

In [144]:
count('hand', 1)

1034

In [153]:
#pos_anchors.remove('hand')
count('hand', 1, pos_anchors[:50])

453

In [151]:
sum(count(w, 1) for w in pos_anchors[:50])

9565

In [163]:
for t, l in zip(texts, labels):
    if l==1 and 'help' in t and len(t)<100:
        print(t)

just added: retail signage to help your store deal with covid 19 concerns
how are rising food prices helping the cdns
covid 19: kind landlord helps tenants stock food in case lock down is announced
please share and help us spreading this message brighton we got you covered 19
how to help second harvest food bank seeks monetary donations preparing for increased demand
could covid19 survivors' blood help save very ill patients?  via
due to bharat band and corona virus many people can t buy food too please help them if you can
coronavirus pandemic could impact the way we shop and help the environment at the same time.
phoenix news covid 19 could help stabilize phoenix area home prices
a helpful summary from our market experts on how is impacting retail amp food service markets
retailers grocery stores offering shopping hours for and to help avoid larger crowds
the numbers are heading in the right direction here s how you can help us pack the pantries
looking to help people during the coro

# Home-spam

In [140]:
for t, l in zip(texts, labels):
    if l==0 and '1 ' in t:
        print(t)

don't know why after 1 use they get scratched by the steak knives! very disappointed. look nice a little thin for my taste
the coffee press says 3 cups, what they really mean is 1 cup. 3 cups refers to 3 4oz cups. what a joke. they are purposefully being dishonest
it worked only 1 day and went dead. it is pretty noisy when you turn it on high due to the fan size. was very disappointed on this one
great tray, however my tray nest in my freezer only holds 1 of these and its a 2 tray nest
for the size of the purifier and the size of the filter, if you smoke in a room it will clear 1 to 2 cigarettes smoke in about an hour...not bad
it isn't really 12 inches but smaller like 11 3/4 inches so very upset. the cast iron pot i have is 12 inches
why, this thing couldn't even play my region 1 dvds. boo urns!
nice color but it didn't work. we couldn't get it to open 1 can so we had to return it. we also thought it might have been used
awful, because 1 side is very thin, sealing different things i 

# Toy-spam

In [124]:
count('would', 0)

1005

In [83]:
count('would', 0, neg_anchors[:50])

920

In [97]:
sum(count(w, 0) for w in neg_anchors[:50])

17260

In [85]:
count('would', 1)

496

In [98]:
count('would', 1, pos_anchors[:50])

436

In [99]:
sum(count(w, 1) for w in pos_anchors[:50])

16790

In [125]:
count('thought', 0)

372

In [130]:
#neg_anchors.remove('thought')
count('thought', 0, neg_anchors[:10])

225

In [128]:
count('thought', 1)

95

In [131]:
count('thought', 1, pos_anchors[:10])

68

In [7]:
for t, l in zip(texts, labels):
    if l==0 and 'one' in t:
        print(t)

it was damaged when i got it not something i would purchase again i kept it after returning one earlier that was more cracked
cheap not the same color truck in the picture and small wish i saved my money the one you get is all blue
this is ok. if you want an expansion worth playing go with one of the full size expansions. i tried this but i don't use it anymore
this toy does not move on carpet & barely on hard flooring. the music is extremely loud with no volume control.don't waste your money
this is a waste of money and does absolutely nothing the description says it does. would never recommend it to anyone
the guitar goes out of tune real easy, but it's great for teaching kids how to tune a guitar. i got one for my little sister and she loves it
when i received the product, the box looked like it may have been opened once before on one side.the tail rotor blade was snapped off on arrival
didn't work right out of the box  save your money. was really hoping this would be a great toy fo